# Import Source data from Kaggle

Import and clean IPO Data
Data set has 3,762 rows, each row representing the IPO company
Over 1,600 columns
The file contains the open, close, low, high, and volume for the 261 trading days following the IPO. (1,305 data points). This data is listed inefficiently in the columns.

First step of data cleaning will be to use the source data to create two tables
    1. A pricing table simply listing the stock ticker and trading day as attribute columns. Stock open, close, low, high, and volume will be listed as the values
    2. An attribute table containing metadata on the stock ticker. I.e. company name, date founded, IPO date, CEO, headquartered location, etc. (all of these data points are on the source file as well


In [125]:
import os
os.getcwd()
os.chdir('C:\\Users\\nmur1\\Google Drive\\Springboard\\Capstone 1\\SourceData')

In [126]:
#import packages
import pandas as pd
import numpy as np
import os

sourcepath = r'C:\Users\nmur1\Google Drive\Springboard\Capstone 1\SourceData'

#import raw data as RawDF
RawDF = pd.read_csv('IPO Data.csv',low_memory = False, encoding='ISO-8859-1')

#break off stockticker and pricing data
Pricing = RawDF.iloc[:,0:1319]
Pricing = Pricing.drop(Pricing.columns[1:9], axis = 1)

#Reindex to symbol 
Pricing = Pricing.set_index('Symbol')

#add Ipo date to dateframe
IPO_Date = RawDF.loc[:,['Symbol','ipoDate']].set_index('Symbol')
IPO_Date['ipoDate'] = pd.to_datetime(IPO_Date['ipoDate'])
Pricing = pd.concat([Pricing,IPO_Date], axis = 1)

#after inspection I foudn that MITT repeated 64 times. Drop the duplicates here
Pricing = Pricing.drop_duplicates()
RawDF.shape

(3762, 1664)

### Pull out daily pricing from the columns to make a Pricing Table
The next sub routine will loop through the pricing dataframe established above and create a new dataframe with the Symbol, Trading Day, Open, Closing, Low, High, and Volume as the column. Row will represent the values for a stock on the given trading day

In [127]:
import datetime as dt

#need to clean data and create a new talbe with ticker, trading day, open price, close price, high, low, and volume
#in columns with the values for in the rows

#create empty lists for my columns
tday = []
Closing = []
High = []
Opening = []
Low = []
Volume = []
tday = []

cols = 0
day = 0

#loop through each column of the dataframe to store the pertenint data
while cols <= 1305:
    
    df = pd.DataFrame(Pricing.index)
    df['trade day'] = day
    tday.append(df)
    Closing.append(Pricing.iloc[:,cols]) #closing price starts at 0
    High.append(Pricing.iloc[:,cols+1]) #high one column over from closing
    Opening.append(Pricing.iloc[:,cols+2]) #opening two columns over from closing
    Low.append(Pricing.iloc[:,cols+3]) # low 3 columns over from closing
    Volume.append(Pricing.iloc[:,cols+4]) #Volume 3 columns over
    
    day = day + 1
    cols = cols + 5 #increment column by 5 (new day is every 5 columns)

#run concatenations on the indexes, to turn into dataframes     
df_Closing = pd.concat(Closing, axis = 0).reset_index()
df_Closing = df_Closing.drop(df_Closing.columns[0], axis = 1)

df_High = pd.concat(High, axis = 0).reset_index()
df_High = df_High.drop(df_High.columns[0], axis = 1)

df_Opening = pd.concat(Opening, axis = 0).reset_index()
df_Opening = df_Opening.drop(df_Opening.columns[0], axis = 1)

df_Low = pd.concat(Low, axis = 0).reset_index()
df_Low = df_Low.drop(df_Low.columns[0], axis = 1)

df_Volume = pd.concat(Volume, axis = 0).reset_index()
df_Volume = df_Volume.drop(df_Volume.columns[0], axis = 1)

df_Day = pd.concat(tday, axis = 0).reset_index()
df_Day = df_Day.drop(df_Day.columns[0], axis = 1)

#concatenate all the above dataframes vertically
df_pricing = pd.concat([df_Day, df_Closing, df_High, df_Opening, df_Low, df_Volume], axis = 1)
df_pricing.columns = ['Symbol', 'Trade Day', 'C', 'H', 'O', 'L', 'V']

#inspect new dataframe
#should have 261 records for each ticker (1 for each trading day)
#drop na's or records that don't have any pricing data
df_pricing.dropna(inplace = True)
df_pricing

,Symbol,Trade Day,C,H,O,L,V
0,A,0,28.6358,33.5207,27.3725,30.6572,59753154.0
1,AAC,0,18.5000,20.1000,17.6000,20.0000,2799073.0
2,AAOI,0,9.9600,10.0900,9.3700,10.0000,948999.0
3,AAP,0,13.9000,14.4667,13.3833,13.4000,371100.0
4,AAT,0,21.2500,22.0000,21.1800,21.5300,15536889.0
...,...,...,...,...,...,...,...
968603,ZNH,261,3.9587,4.0000,3.8753,3.8753,98700.0
968606,ZSAN,261,45.6000,45.6000,42.8000,42.8000,21.0
968607,ZTO,261,15.7300,15.8950,15.3500,15.6000,1922801.0
968611,ZX,261,3.4600,3.5500,3.3600,3.5300,22850.0


In [128]:
#Save CleanData set to new folder
os.chdir('C:\\Users\\nmur1\\Google Drive\\Springboard\\Capstone 1\\CleanData')
df_pricing.to_csv('DailyPricing.csv')

# Make Attribute Table with Key MetaData Points

My next step will be to create an attribute or metadata table containing all of the key stats on the stock company. There will be a record/row for each stock with multiple data points including date founded, revenue, income, ipoDate, day of week IPO'd, State, etc

In [129]:
#make my attribute table with the other metadata
#metadata is in columns 0 through 9 and all columns from 1319 to the end of the dataset. 
#the columns in the middle were pricing data points that I split off into the pricing table listed above

#split out desired columns using iloc. Concatenate back to one and inspect:
Attribute1 = RawDF.iloc[:,0:9]
Attribute2 = RawDF.iloc[:,1319:]
FullAttribute = pd.concat([Attribute1,Attribute2], axis = 1)
FullAttribute.shape

(3762, 354)

# Reduce Size of Metadata table by dropping irrelevant columns

After analysis my Attribute table still had 354 columns which was quite cumbersome. I reviewed the columns and found that the majority had N/A. To start the below code will identify columsn and % of N/A values. To start I'm only going to keep attributes with less than 50% N/A values. This leaves 50 columns wiht most of the key metadata I'm looking for. I.e. IPO date, CEO age/gender, company location, company age at IPO, year founded etc.

In [130]:
#create df with N/A percentages
nas=pd.DataFrame(FullAttribute.isnull().sum().sort_values(ascending=True)/len(FullAttribute),columns = ['percent'])

#filter percent less than 50
nasFilt = nas['percent']<=.5

#create and apply boolean series filter
tokeep = nas[nasFilt]
df_Att =FullAttribute.loc[:,tokeep.index]

#inspect data
df_Att.to_clipboard()
df_Att.shape



(3762, 50)

# Inspect the number of unique values in each column

In [131]:
df_Att.nunique()

Symbol                        3699
Safe                             2
Profitable                       2
yearDifferenceGrouped            1
FoundingDateGrouped              6
usablePresidentGender            8
usableCEOGender                  8
FiscalMonth                     13
USACompany                       3
MarketYearTrend               2144
Market6MonthTrend             2144
Market3MonthTrend             2144
MarketMonthTrend              2144
ipoDate                       2239
Summary Quote                 3700
HomeRun                          2
Month                           12
MarketCap                     3552
Name                          3556
dayOfWeek                        5
Day                             31
Year                            23
daysProfitGrouped                5
daysProfit                     263
DaysBetterThanSP               153
LastSale                      2756
usablePresidentAge               8
usableCEOAge                     8
CEOGender           

### Gender contains 8 values - need to fix that

In [132]:
df_Att['usableCEOGender'].value_counts()

Blank            1971
Unknown           855
male              763
unknown            81
female             44
mostly_male        21
andy               16
mostly_female      11
Name: usableCEOGender, dtype: int64

In [133]:
df_Att['usableCEOGender'] = df_Att['usableCEOGender'].str.replace('Unknown', 'unknown')
df_Att['usableCEOGender'] = df_Att['usableCEOGender'].str.replace('Blank', 'unknown')
df_Att['usableCEOGender'] = df_Att['usableCEOGender'].str.replace('mostly_male', 'male')
df_Att['usableCEOGender'] = df_Att['usableCEOGender'].str.replace('mostly_female', 'female')
df_Att['usableCEOGender'] = df_Att['usableCEOGender'].str.replace('andy', 'male')

In [134]:
df_Att['usableCEOGender'].value_counts()

unknown    2907
male        800
female       55
Name: usableCEOGender, dtype: int64

In [135]:
#CEOGender field looks to contain more relevant data then usableCEOGender

df_Att.drop(columns = 'usableCEOGender',inplace = True)

Fixed misidentified genders in other CEO Gender column

In [136]:
df_Att['CEOGender'].value_counts()

male             2683
unknown           283
female            155
mostly_male        57
andy               51
mostly_female      34
Name: CEOGender, dtype: int64

In [137]:
df_Att['CEOGender'] = df_Att['CEOGender'].str.replace('Unknown', 'unknown')
df_Att['CEOGender'] = df_Att['CEOGender'].str.replace('Blank', 'unknown')
df_Att['CEOGender'] = df_Att['CEOGender'].str.replace('mostly_male', 'male')
df_Att['CEOGender'] = df_Att['CEOGender'].str.replace('mostly_female', 'female')
df_Att['CEOGender'] = df_Att['CEOGender'].str.replace('andy', 'male')
df_Att['CEOGender'].value_counts()

male       2791
unknown     283
female      189
Name: CEOGender, dtype: int64

## Drop Other Unnecessary Columns

In [147]:
todrop = ['usablePresidentGender', 'PresidentGender', 'usablePresidentAge', 'HomeRun', 'usableCEOAge','PresidentName','CEOInChargeDuringIPO','CEOTakeOver',
             'PresidentAge','presidentInChargeDuringIPO','PresidentTakeOver','Safe','yearDifferenceGrouped', 'daysProfitGrouped',
                'daysProfit', 'DaysBetterThanSP', 'FoundingDateGrouped' ]

for d in todrop:
    
    try:
        df_Att.drop(columns = d, inplace = True)
    except:
        df_Att

df_Att.nunique()

Symbol               2249
Profitable              2
FiscalMonth            13
USACompany              1
MarketYearTrend      1532
Market6MonthTrend    1532
Market3MonthTrend    1532
MarketMonthTrend     1532
ipoDate              1612
Summary Quote        2250
Month                  12
MarketCap            2173
Name                 2170
dayOfWeek               5
Day                    31
Year                   23
LastSale             1840
CEOGender               3
CEOName              2074
Industry              127
Sector                 12
YearFounded           139
exactDateFounded     1335
yearDifference        151
CEOAge                 54
employees            1214
employeesGrouped        7
FiscalDateEnd          23
City                  692
stateCountry           52
netIncome            2059
Revenue              1762
Country                 1
Revenue_M            1762
Income_M             2056
DayofWeek               5
dtype: int64

# Inspect the Data I Dropped to Determine if I want to Source Elsewhere

After significantly paring down my attribute table I reviewed all of the fields that were greater than 50% N/A to see if there's anything I wanted to keep. Most are key financila metrics for the company pre-IPO. My hypothesis is that these metrics would have an impact on pricing performance so I will need to find another datasource to pull those metrics in. For purposes of this exercise I'm not going to pull in all 304 rows but we'll start with the big ones:

Pre IPO Revenue ||
Pre IPO EBIDTA ||
Pre IPO Cash ||

I don't have a datasource for those yet so let's clean up the table we have and add a few functions in the next two steps

In [139]:
NullColumns =pd.DataFrame(FullAttribute.isnull().sum().sort_values(ascending=False)/len(FullAttribute),columns = ['percent'])
NullFilt = NullColumns['percent']>.50

NullColumns[NullFilt]

,percent
Other_intangiblesYearBeforeIPO,0.999734
Loans_issuedYearBeforeIPO,0.999734
Preferred_dividendsYearBeforeIPO,0.999734
Restricted_cash_and_cash_equivalentsYearBeforeIPO,0.999734
Provision_for_loan_lossesYearBeforeIPO,0.999734
...,...
Net_cash_provided_by_operating_activitiesYearBeforeIPO,0.891015
Net_incomeYearBeforeIPO,0.876396
Fiscal_year_ends_in_December_USDYearBeforeIPO,0.865231
exactDiffernce,0.583200


### Run Additional Conversion Functions to Clean Metadata

In [140]:
#create an attribute table for additional analysis
#pd.options.display.float_format = '{:.5f}'.format
#define US or Other country
def country(x):
    
    if len(str(x).strip()) == 2:
        return 'US'
    else:
        return 'Other'
        
#Revenue and Income columns end with 'B' to denote billions, 'M' to denote millions, or have the straight number if
# less than 1 million. The below function will stip the last character and convert to a float value that consistently
# represents revenue and income has millions. I.e. 1 billion displayed as 1,000; 1 million displayed as 1; 100,000 displayed
#as .1

def conversion(x):
   
    s = str(x).strip() #ensure there are no spaces in string

    suffix = str(x)[-1] #get last charcter
    
    if suffix == 'B': #if B define multiple as 1,000
        mult = 1000
    elif suffix == 'M': #if M define multiple as 1
        mult = 1
    else:
        mult = 0.000001 #if not B or M multiple is 1/1000000
 
    
    #loop through stirng and remove non numbers. Mainly $ signs and commas
    #noticed that some strings also had typos with parentheticals so 
    #the below loop ensures that all non numbers except for decimial points and negative symbols
    #are removed
    
    for letter in s:
         if letter.isdigit() == False and letter != '.' and letter != "-":
            s = s.replace(letter,"")
    
    #handle the nulls
    if pd.isnull(x) == True:
        s = 0
    
    #convert the final string to a float and multiple by the multiple
    return round(float(s) * mult, 3)

def DOW(x):

    day = ['Mon','Tue','Wed','Thur','Fri','Sat','Sun']
    return day[x.weekday()]
        

df_Att['Country'] = df_Att['stateCountry'].apply(country)
df_Att['Revenue_M'] = df_Att['Revenue'].apply(conversion)
df_Att['Income_M'] = df_Att['netIncome'].apply(conversion)
df_Att.ipoDate = pd.to_datetime(df_Att['ipoDate'])
df_Att['DayofWeek'] = df_Att['ipoDate'].apply(DOW) #Add the name of the day ipo'd
df_Att = df_Att[df_Att.Country == 'US']# Filter on US
df_Att.sort_values('Revenue_M')#sort by revenue
df_Att.to_excel('IPO Attributes.xls') #export to excel for review/analysis

df_Att


,Symbol,Profitable,yearDifferenceGrouped,FoundingDateGrouped,FiscalMonth,USACompany,MarketYearTrend,Market6MonthTrend,Market3MonthTrend,MarketMonthTrend,...,employeesGrouped,FiscalDateEnd,City,stateCountry,netIncome,Revenue,Country,Revenue_M,Income_M,DayofWeek
0,A,1,Unknown,1951 - 2000,Oct,Yes,2.039844,2.312974,2.352508,1.601165,...,>10000,18-Oct,Santa Clara,CA,$684.00M,$4.47B,US,4470.00,684.00,Thur
1,AAC,1,Unknown,2011 - present,Dec,Yes,0.881839,0.138536,-1.194498,-2.452645,...,1001 - 5000,18-Dec,Brentwood,TN,$-20.58M,$317.64M,US,317.64,-20.58,Thur
2,AAOI,1,Unknown,1951 - 2000,Dec,Yes,1.443672,1.286165,0.926398,0.761732,...,1001 - 5000,18-Dec,Sugar Land,TX,$73.95M,$382.33M,US,382.33,73.95,Thur
3,AAP,1,Unknown,1901 - 1950,Dec,Yes,-0.745906,-0.128110,1.153716,0.613550,...,>10000,18-Dec,Roanoke,VA,$475.51M,$9.37B,US,9370.00,475.51,Thur
4,AAT,0,Unknown,2001 - 2010,Dec,Yes,2.263666,1.813736,1.824305,1.692499,...,101 - 500,18-Dec,San Diego,CA,$29.08M,$311.68M,US,311.68,29.08,Thur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,ZSAN,0,Unknown,2011 - present,Dec,Yes,1.055378,0.515719,-0.448387,-0.373177,...,0 - 100,18-Dec,Fremont,CA,$-29.11M,NaN,US,0.00,-29.11,Tue
3756,ZTS,1,Unknown,2011 - present,Dec,Yes,2.456595,2.497749,1.941604,1.830451,...,5001 - 10000,18-Dec,New York,NY,$864.00M,$5.31B,US,5310.00,864.00,Fri
3757,ZUMZ,1,Unknown,1951 - 2000,Jan,Yes,0.546793,-0.792605,-0.612410,0.510960,...,5001 - 10000,19-Jan,Lynnwood,WA,$26.80M,$927.40M,US,927.40,26.80,Fri
3758,ZUO,0,Unknown,2001 - 2010,Jan,Yes,0.794603,-0.113986,-0.734112,0.031797,...,501 - 1000,19-Jan,Redwood City,CA,$-47.16M,$167.93M,US,167.93,-47.16,Thur


In [141]:
#save attributes to new csv file
df_Att.to_csv('IPO Attributes.csv')

In [142]:
#print min and max ipoDates

print(df_Att.ipoDate.min())
print(df_Att.ipoDate.max())

1996-01-05 00:00:00
2018-04-13 00:00:00


My initial data collection and cleaning phase is complete. I now have a much more managable set of data that's conducive to analysis. A clean pricing table with pricing laid out by day and company in the rows and low, high, opening, closing, and volume in the columns. 

My attribute table has my stocks filtered out by US company and each stock has 50 pertient meta data points



Next Steps:
    
• Import MacroEconomic for all months/years relevant to IPO's. I'll start with gdp growth, unemployment, and interest rates I plan to use Pandas DataReader function which has a direct link to https://fred.stlouisfed.org

• I'll need to do some more digging to see where I can find the pre-IPO Revenue, Cash, and EBITDA metrics I mentioned in the steps above to add to my attribute table 

# Import GDP, Fed Funds Rate, Unemployment, Consumer Sentiment

Pandas DataReader package has a great way to improt macro-economic data directly from the fred database
make sure you have the data reader package installed on your pc: 
conda install -c anaconda pandas-datareader in the anaconda prompt download for you

In [143]:
from pandas_datareader.data import DataReader
from datetime import date
start = date(1990,1,1)

#import GDP and calculate quarterly growth
GDP = DataReader('GDPC1', 'fred', start )
GDP['growth'] = GDP.GDPC1.pct_change() * 100
GDP = GDP.reset_index()
GDP.columns = ['DATE', 'GDP', 'GDP Growh']

#import Fed Funds Interest rate
EFFR = DataReader('FEDFUNDS', 'fred',start).reset_index()

#import unemployment rate
UNRATE = DataReader('UNRATE', 'fred',start).reset_index()

#import Consumer Sentiment Score
CS = DataReader('UMCSENT', 'fred', start).reset_index()

#merge dataframes to one
Macro = pd.merge(GDP, EFFR, on = 'DATE', how = 'left')
Macro = pd.merge(Macro, UNRATE, on = 'DATE', how = 'left')
Macro = pd.merge(Macro, CS, on = 'DATE', how = 'left')


#export table to my clean data folder
os.getcwd()
os.chdir('C:\\Users\\nmur1\\Google Drive\\Springboard\\Capstone 1\\CleanData')
Macro.to_csv('MacroEcon.csv')